# Instalando dependências

In [ ]:
from IPython.display import clear_output
import sys

IN_COLAB = 'google.colab' in sys.modules

In [ ]:
if IN_COLAB:
    !git clone https://github.com/LucaLemos/UFRPE_AprendizagemReforco
    sys.path.append("/content/UFRPE_AprendizagemReforco")

    clear_output()
else:
    from os import path
    sys.path.append( path.dirname( path.dirname( path.abspath("__main__") ) ) )

In [ ]:
if IN_COLAB:
    # for saving videos
    !apt-get install ffmpeg
    !pip install gymnasium==1.0.0   # conferir se precisa
    #!pip install tianshou # Para criar o Replay_Buffer
    #!pip install d3rlpy==2.7.0
    # clone repository

# Criando Dataset

In [1]:
import gymnasium as gym
import torch
from util.algorithms import run_sarsa
from util.network import ReplayBuffer, MLP, epsilon_greedy_qnet, calc_loss
from IPython.display import clear_output

In [2]:
DATASET_SIZE = 200_000  # Tamanho do conjunto de dados (replay buffer)
LEARNING_RATE = 1e-3  # Taxa de aprendizado para o otimizador
GAMMA = 0.99  # Fator de desconto
BATCH_SIZE = 128  # Tamanho do batch para treinamento da rede neural

In [3]:
ENV_NAMES = []
ENVS_REPLAY_BUFFER = []

## Dataset Básico

In [4]:
# Passo 1: Coletar um conjunto fixo de transições (Replay Buffer)
# Ambientes Básicos
ENV_NAMES += ["FrozenLake-v1", "Taxi-v3", "CliffWalking-v0"]
for i, env_name in enumerate(ENV_NAMES):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    replay_buffer = ReplayBuffer(DATASET_SIZE, BATCH_SIZE, device)
    env = gym.make(env_name, render_mode="rgb_array")
    
    sum_rewards_per_ep, q = run_sarsa(env, replay_buffer, DATASET_SIZE, LEARNING_RATE, GAMMA)
    ENVS_REPLAY_BUFFER.append((env_name, env, replay_buffer, sum_rewards_per_ep, q))
    replay_buffer.save_config(f"config\dataset\sarsa\{env_name}.json")


Run sarsa in <TimeLimit<OrderEnforcing<PassiveEnvChecker<FrozenLakeEnv<FrozenLake-v1>>>>>
Episódio 0 terminou com recompensa 0.0 na transição 3
Episódio 100 terminou com recompensa 0.0 na transição 1601
Episódio 200 terminou com recompensa 0.0 na transição 3020
Episódio 300 terminou com recompensa 0.0 na transição 4652
Episódio 400 terminou com recompensa 0.0 na transição 6299
Episódio 500 terminou com recompensa 0.0 na transição 7809
Episódio 600 terminou com recompensa 0.0 na transição 9291
Episódio 700 terminou com recompensa 0.0 na transição 10720
Episódio 800 terminou com recompensa 0.0 na transição 12090
Episódio 900 terminou com recompensa 0.0 na transição 13643
Episódio 1000 terminou com recompensa 0.0 na transição 15112
Episódio 1100 terminou com recompensa 0.0 na transição 16545
Episódio 1200 terminou com recompensa 0.0 na transição 18010
Episódio 1300 terminou com recompensa 0.0 na transição 19488
Episódio 1400 terminou com recompensa 0.0 na transição 21061
Episódio 1500 ter

## Dataset com Rede Neural

In [ ]:
from torch import optim, nn
import numpy as np

In [ ]:
def DQN_TRAIN(env, env_name, gamma, qnet, qnet_lr, target_qnet, target_update_freq, replay_buffer, batch_size, epsilon_f, epsilon_decay_period, transitions):
    # Cria o otimizador, que vai fazer o ajuste dos pesos da 'qnet',
    # Usa uma técnica de gradiente descendente de destaque, chamada ADAM
    optimizer = optim.Adam(qnet.parameters(), lr=qnet_lr)

    episode_reward_list = []
    step = 0
    epsilon = 1.0

    state, _ = env.reset()
    episode_reward = 0.0

    for _ in range(transitions):
        # Decaimento linear do epsilon
        epsilon = max(epsilon_f, 1.0 - step / epsilon_decay_period)

        action = epsilon_greedy_qnet(qnet, env, state, epsilon)

        # Faz um passo / Aplica uma ação no ambiente
        new_state, reward, terminated, truncated, _ = env.step(action)

        step += 1
        done = terminated or truncated
        episode_reward += reward

        # Adiciona no buffer
        replay_buffer.add(state, action, reward, terminated, new_state)
        state = new_state

        if done:
            episode_reward_list.append(episode_reward)
            if len(episode_reward_list) % 100 == 0:
                mean_reward = np.mean(episode_reward_list[-100:])
                print(f'Episode Reward: {episode_reward_list[-1]} | Epsilon: {epsilon} | Step: {step} | Mean Reward: {mean_reward}')
            state, _ = env.reset()
            episode_reward = 0.0

            # Abaixo, faz vários loggings de dados
            
        # Faz a 'tgt_net' receber os mesmos valores de pesos da 'qnet', na frequência indicada
        
        if step > batch_size:
            if step % target_update_freq == 0:
                target_qnet.load_state_dict(qnet.state_dict())
            # Escolhe amostras aleatórios do buffer e faz uma atualização dos pesos da rede
            optimizer.zero_grad()
            batch = replay_buffer.sample_continuous()
            loss_t = calc_loss(batch, qnet, target_qnet, gamma)
            loss_t.backward()
            optimizer.step()
            
    mean_reward = np.mean(episode_reward_list[-100:])
    print(f"Mean_reward: {mean_reward}")
    filename = f"./neural_dataset/{env_name}.pth"
    torch.save(qnet.state_dict(), filename)
    print(f"Model saved as {filename}")

In [ ]:
GOAL_REWARD = 200
GAMMA = 0.999
REPLAY_SIZE = 100_000
BATCH_SIZE = 32
LEARNING_RATE = 1e-3
SYNC_TARGET_FRAMES = 1_000

EPSILON_DECAY_PERIOD = 75_000
EPSILON_FINAL = 2e-2

In [ ]:
#ENV_NAMES += ["CartPole-v1"]
ENV_NAMES += ["LunarLander-v3"]
for i, env_name in enumerate(ENV_NAMES[-1:]):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    #replay_buffer = ReplayBuffer(DATASET_SIZE, BATCH_SIZE, device)
    buffer = ReplayBuffer(DATASET_SIZE, BATCH_SIZE, device)
    print(env_name)
    env = gym.make(env_name)
    
    qnet1 = MLP(env.observation_space.shape[0], env.action_space.n, [256,256])
    qtarget1 = MLP(env.observation_space.shape[0], env.action_space.n, [256,256])

    DQN_TRAIN(
    env = env,
    env_name = env_name,
    gamma = GAMMA,
    qnet = qnet1,
    qnet_lr = LEARNING_RATE,
    target_qnet = qtarget1,
    target_update_freq = SYNC_TARGET_FRAMES,
    replay_buffer= buffer,
    batch_size = BATCH_SIZE,
    epsilon_f = EPSILON_FINAL,
    epsilon_decay_period = EPSILON_DECAY_PERIOD,
    transitions = REPLAY_SIZE)

    #!sum_rewards_per_ep, q = run_sarsa(env, replay_buffer, DATASET_SIZE, LEARNING_RATE, GAMMA)
    ENVS_REPLAY_BUFFER.append((env_name, env, buffer))
    buffer.save_config(f"config\dataset\dqn\{env_name}.json")

# Treinando o Modelo

## Carregando Datasets

In [4]:
import gymnasium as gym
from util.network import ReplayBuffer


In [5]:
ENV_NAMES = ["FrozenLake-v1", "Taxi-v3", "CliffWalking-v0", "CartPole-v1", "LunarLander-v3"]

In [6]:
ENVS_REPLAY_BUFFER = []
for env_name in ENV_NAMES[:3]:
    replay_buffer = ReplayBuffer.load_config(f"config\dataset\sarsa\{env_name}.json")
    env = gym.make(env_name)
    ENVS_REPLAY_BUFFER.append((env_name, env, replay_buffer))
for env_name in ENV_NAMES[3:]:
    replay_buffer = ReplayBuffer.load_config(f"config\dataset\dqn\{env_name}.json")
    env = gym.make(env_name)
    ENVS_REPLAY_BUFFER.append((env_name, env, replay_buffer))

## Definições de Treino

In [7]:
import argparse

In [8]:
def get_config(env_name, count_episodes, seed, gamma, tau, lr, steps, hidden_size, isDiscrete):
    parser = argparse.ArgumentParser(description='RL')
    parser.add_argument("--run_name", type=str, default=f"{env_name}-DQN-FQI", help="Run name, default: FQI-DQN")
    parser.add_argument("--env", type=str, default=env_name, help="Gym environment name, default: CartPole-v0")
    parser.add_argument("--episodes", type=int, default=count_episodes, help="Number of episodes, default: 200")
    parser.add_argument("--seed", type=int, default=seed, help="Seed, default: 1")
    parser.add_argument("--steps", type=int, default=steps, help="Saves the network every x epochs, default: 25")
    
    parser.add_argument("--gamma", type=float, default=gamma, help="Saves the network every x epochs, default: 25")
    parser.add_argument("--tau", type=float, default=tau, help="Saves the network every x epochs, default: 25")
    parser.add_argument("--lr", type=float, default=lr, help="Saves the network every x epochs, default: 25")
    parser.add_argument("--hidden_size", type=list[int], default=hidden_size, help="Saves the network every x epochs, default: 25")
    parser.add_argument("--isDiscrete", type=bool, default=isDiscrete, help="Saves the network every x epochs, default: 25")
    
    args, _ = parser.parse_known_args()
    return args

In [9]:
from util.network import FQIAgent, to_one_hot, MLP
import numpy as np
import random
import torch
from collections import deque

In [10]:
def train_DQN_FQI(config, buffer):
    np.random.seed(config.seed)
    random.seed(config.seed)
    torch.manual_seed(config.seed)
    env = gym.make(config.env)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    losses = deque(maxlen=1000)
    if config.isDiscrete:
        agent = FQIAgent(env.observation_space.n, env.action_space.n, config.tau, config.gamma, config.lr, config.hidden_size, device, config.isDiscrete)
    else:
        agent = FQIAgent(env.observation_space.shape[0], env.action_space.n, config.tau, config.gamma, config.lr, config.hidden_size, device, config.isDiscrete)
    for i in range(config.steps):
        loss = agent.learn(buffer.sample_continuous())
        losses.append(loss)     
        if i % 1000 == 0:
            print(f"[TRAIN {i}] Mean Q Loss: {np.mean(losses)} ")
    
    agent.save(config, save_name="FQI-DQN")
    
    return env

In [11]:
def evaluate(config, env, model):
    total_rewards = []
    num_episodes = config.episodes
    for i in range(num_episodes):
        print(f"[Episódio {i}]")
        state, _ = env.reset()
        done = False
        episode_reward = 0
        step = 0
        while not done:
            state_tensor = torch.tensor([state], dtype=torch.long)
            state_tensor = to_one_hot(model.state_size, state_tensor)[0].float().unsqueeze(0)
            with torch.no_grad():
                q_values = model(state_tensor)
                action = torch.argmax(q_values).item()
            state, reward, done, truncated, info = env.step(action)
            episode_reward += reward
            step += 1
            if step == 10000:
                break
        total_rewards.append(episode_reward)

    print(f"Média de recompensa após {num_episodes} episódios: {sum(total_rewards) / num_episodes}")

In [12]:
# Função para extrair a tabela Q da rede neural
def extract_q_table(q_network, num_states, num_actions):
    q_table = np.zeros((num_states, num_actions))
    for state in range(num_states):
        state_tensor = torch.tensor([state], dtype=torch.long)
        state_tensor = to_one_hot(num_states, state_tensor)[0].float().unsqueeze(0)
        q_values = q_network(state_tensor).detach().numpy()
        q_table[state] = q_values
    return q_table

In [13]:
from util.qtable_helper import record_video_qtable
from util.notebook import display_videos_from_path

In [14]:

def extract_and_display(model, env_name):
    # Extrair a tabela Q da rede neural treinada
    q_table = extract_q_table(model, model.state_size, model.action_size)

    # Gravar um vídeo da política treinada
    video_path = 'videos/'  # Pasta onde os vídeos serão salvos
    video_prefix = f"fqi_{env_name}"  # Prefixo para o nome do arquivo de vídeo

    # Gravar o vídeo
    record_video_qtable(env_name, q_table, episodes=2, folder=video_path, prefix=video_prefix)

    # Exibir o vídeo gravado
    display_videos_from_path(video_path, prefix=video_prefix)

In [15]:
def evaluate_Continuous(config, env, model):
    total_rewards = []
    num_episodes = config.episodes
    for i in range(num_episodes):
        print(f"[Episódio {i}]")
        state, _ = env.reset()
        done = False
        episode_reward = 0.0
        step = 0
        while not done:
            state_v = torch.tensor(state, dtype=torch.float32)
            state_v = state_v.unsqueeze(0)  # Adiciona dimensão de batch como eixo 0 (e.g. transforma uma lista [a,b,c] em [[a,b,c]])
            with torch.no_grad():
                q_vals_v = model(state_v)
                _, act_v = torch.max(q_vals_v, dim=1)
            action = int(act_v.item())


            state, reward, done, truncated, info = env.step(action)
            episode_reward += reward
            step += 1
            if step == 10000:
                break
        total_rewards.append(episode_reward)

    print(f"Média de recompensa após {num_episodes} episódios: {sum(total_rewards) / num_episodes}")

In [16]:
# Função para renderizar o ambiente com o modelo treinado
import time
def render_environment(model, env_name, device="cpu", episodes=5, max_steps=100000):
    env = gym.make(env_name, render_mode="human")  # Modo "human" para renderização
    for episode in range(episodes):
        state, _ = env.reset()
        done = False
        episode_reward = 0
        step = 0

        while not done and step < max_steps:
            # Renderiza o ambiente
            env.render()
            time.sleep(0.02)  # Adiciona um pequeno delay para visualização

            # Converte o estado para um tensor e passa pelo modelo
            state_tensor = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(device)
            with torch.no_grad():
                q_values = model(state_tensor)
                action = torch.argmax(q_values).item()

            # Executa a ação no ambiente
            state, reward, done, truncated, info = env.step(action)
            episode_reward += reward
            step += 1

            if done or truncated:
                print(f"Episódio {episode + 1}: Recompensa = {episode_reward}, Passos = {step}")
                break

    env.close()

## FrozenLake

In [17]:
COUNT_EPISODES = 50
SEED = 777
BATCH_SIZE = 32
STEPS = 100_000
GAMMA = 0.95 # Fator de desconto
TAU = 1e-1      # Taxa de atualização da target_network

LEARNING_RATE = 3e-4  # Taxa de aprendizado para o otimizador
HIDDEN_SIZE = [64, 32]
ISDISCRETE = True
# Extra:

In [18]:
ENVS_REPLAY_BUFFER[0][2].batch_size = BATCH_SIZE

In [19]:
config = get_config(ENVS_REPLAY_BUFFER[0][0], COUNT_EPISODES, SEED, GAMMA, TAU, LEARNING_RATE, STEPS, HIDDEN_SIZE, ISDISCRETE)

In [20]:
env = train_DQN_FQI(config, ENVS_REPLAY_BUFFER[0][2])

[TRAIN 0] Mean Q Loss: 0.02159975655376911 
[TRAIN 1000] Mean Q Loss: 0.001326863380440045 
[TRAIN 2000] Mean Q Loss: 0.0010751593966651853 
[TRAIN 3000] Mean Q Loss: 0.0012115980252929148 
[TRAIN 4000] Mean Q Loss: 0.0012711787573389303 
[TRAIN 5000] Mean Q Loss: 0.0014905044858533074 
[TRAIN 6000] Mean Q Loss: 0.0014968430143599107 
[TRAIN 7000] Mean Q Loss: 0.0015571928037788894 
[TRAIN 8000] Mean Q Loss: 0.0016794220346127986 
[TRAIN 9000] Mean Q Loss: 0.0016714792748971376 
[TRAIN 10000] Mean Q Loss: 0.0018782683907775208 
[TRAIN 11000] Mean Q Loss: 0.0017532436412875541 
[TRAIN 12000] Mean Q Loss: 0.0015765380987941171 
[TRAIN 13000] Mean Q Loss: 0.0020959682068059917 
[TRAIN 14000] Mean Q Loss: 0.002356295665114885 
[TRAIN 15000] Mean Q Loss: 0.0017691183831484523 
[TRAIN 16000] Mean Q Loss: 0.0021328804709919496 
[TRAIN 17000] Mean Q Loss: 0.001881238216519705 
[TRAIN 18000] Mean Q Loss: 0.0017130367566642235 
[TRAIN 19000] Mean Q Loss: 0.0018195560807798756 
[TRAIN 20000] Mean

In [21]:
env = ENVS_REPLAY_BUFFER[0][1]
model = MLP(env.observation_space.n, env.action_space.n, HIDDEN_SIZE)
model.load_state_dict(torch.load(f"trained_models/{ENVS_REPLAY_BUFFER[0][0]}FQI-DQN.pth"))
model.eval()  # Definir para modo de avaliação

MLP(
  (layers): Sequential(
    (0): Linear(in_features=16, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=4, bias=True)
  )
)

In [22]:
evaluate(config, env, model)

[Episódio 0]
[Episódio 1]
[Episódio 2]
[Episódio 3]
[Episódio 4]
[Episódio 5]
[Episódio 6]
[Episódio 7]
[Episódio 8]
[Episódio 9]
[Episódio 10]
[Episódio 11]
[Episódio 12]
[Episódio 13]
[Episódio 14]
[Episódio 15]
[Episódio 16]
[Episódio 17]
[Episódio 18]
[Episódio 19]
[Episódio 20]
[Episódio 21]
[Episódio 22]
[Episódio 23]
[Episódio 24]
[Episódio 25]
[Episódio 26]
[Episódio 27]
[Episódio 28]
[Episódio 29]
[Episódio 30]
[Episódio 31]
[Episódio 32]
[Episódio 33]
[Episódio 34]
[Episódio 35]
[Episódio 36]
[Episódio 37]
[Episódio 38]
[Episódio 39]
[Episódio 40]
[Episódio 41]
[Episódio 42]
[Episódio 43]
[Episódio 44]
[Episódio 45]
[Episódio 46]
[Episódio 47]
[Episódio 48]
[Episódio 49]
Média de recompensa após 50 episódios: 0.72


d:\Downloads_D\Faculdade\git\util\network.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  states = torch.tensor(states, dtype=torch.long)  # Garante que é tensor de inteiros


In [23]:
extract_and_display(model, ENVS_REPLAY_BUFFER[0][0])

d:\Downloads_D\Faculdade\git\.conda\lib\site-packages\gymnasium\wrappers\rendering.py:283: UserWarning: WARN: Overwriting existing videos at d:\Downloads_D\Faculdade\git\videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


## Taxi

In [24]:
COUNT_EPISODES = 50
SEED = 777
BATCH_SIZE = 256
STEPS = 300_000
GAMMA = 0.97  # Fator de desconto
TAU = 1e-2      # Taxa de atualização da target_network
LEARNING_RATE = 5e-4  # Taxa de aprendizado para o otimizador
HIDDEN_SIZE = [512, 256]
ISDISCRETE = True
# Extra:

In [25]:
ENVS_REPLAY_BUFFER[1][2].batch_size = BATCH_SIZE

In [26]:
config = get_config(ENVS_REPLAY_BUFFER[1][0], COUNT_EPISODES, SEED, GAMMA, TAU, LEARNING_RATE, STEPS, HIDDEN_SIZE, ISDISCRETE)

In [27]:
env = train_DQN_FQI(config, ENVS_REPLAY_BUFFER[1][2])

[TRAIN 0] Mean Q Loss: 0.944583535194397 
[TRAIN 1000] Mean Q Loss: 0.1101786213084124 
[TRAIN 2000] Mean Q Loss: 0.028804151193005965 
[TRAIN 3000] Mean Q Loss: 0.018035940087866038 
[TRAIN 4000] Mean Q Loss: 0.012767822462134064 
[TRAIN 5000] Mean Q Loss: 0.006072314217453823 
[TRAIN 6000] Mean Q Loss: 0.004921916572435294 
[TRAIN 7000] Mean Q Loss: 0.0028710455946275035 
[TRAIN 8000] Mean Q Loss: 0.0019924480461049826 
[TRAIN 9000] Mean Q Loss: 0.0014881917492020876 
[TRAIN 10000] Mean Q Loss: 0.0013500999240786769 
[TRAIN 11000] Mean Q Loss: 0.0014599549602717162 
[TRAIN 12000] Mean Q Loss: 0.001293285060848575 
[TRAIN 13000] Mean Q Loss: 0.0012814313532144297 
[TRAIN 14000] Mean Q Loss: 0.0012390523571521044 
[TRAIN 15000] Mean Q Loss: 0.0012574146758415736 
[TRAIN 16000] Mean Q Loss: 0.0010472098756581544 
[TRAIN 17000] Mean Q Loss: 0.0011482152615790255 
[TRAIN 18000] Mean Q Loss: 0.0011698885162477381 
[TRAIN 19000] Mean Q Loss: 0.0008515412345004733 
[TRAIN 20000] Mean Q Loss:

In [28]:
env = ENVS_REPLAY_BUFFER[1][1]
model = MLP(env.observation_space.n, env.action_space.n, HIDDEN_SIZE)
model.load_state_dict(torch.load(f"trained_models/{ENVS_REPLAY_BUFFER[1][0]}FQI-DQN.pth"))
model.eval()  # Definir para modo de avaliação

MLP(
  (layers): Sequential(
    (0): Linear(in_features=500, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=6, bias=True)
  )
)

In [29]:
evaluate(config, env, model)

[Episódio 0]
[Episódio 1]
[Episódio 2]
[Episódio 3]
[Episódio 4]
[Episódio 5]
[Episódio 6]
[Episódio 7]
[Episódio 8]
[Episódio 9]
[Episódio 10]
[Episódio 11]
[Episódio 12]
[Episódio 13]
[Episódio 14]
[Episódio 15]
[Episódio 16]
[Episódio 17]
[Episódio 18]
[Episódio 19]
[Episódio 20]
[Episódio 21]
[Episódio 22]
[Episódio 23]
[Episódio 24]
[Episódio 25]
[Episódio 26]
[Episódio 27]
[Episódio 28]
[Episódio 29]
[Episódio 30]
[Episódio 31]
[Episódio 32]
[Episódio 33]
[Episódio 34]
[Episódio 35]
[Episódio 36]
[Episódio 37]
[Episódio 38]
[Episódio 39]
[Episódio 40]
[Episódio 41]
[Episódio 42]
[Episódio 43]
[Episódio 44]
[Episódio 45]
[Episódio 46]
[Episódio 47]
[Episódio 48]
[Episódio 49]
Média de recompensa após 50 episódios: 7.2


In [30]:
extract_and_display(model, ENVS_REPLAY_BUFFER[1][0])

## Cliffwalking

In [31]:
COUNT_EPISODES = 50
SEED = 777
BATCH_SIZE = 128
STEPS = 200_000
GAMMA = 0.98  # Fator de desconto
TAU = 5e-3      # Taxa de atualização da target_network
LEARNING_RATE = 5e-4  # Taxa de aprendizado para o otimizador
HIDDEN_SIZE = [256, 128, 64]  # Tamanho da camada oculta
ISDISCRETE = True
# Extra:

In [32]:
ENVS_REPLAY_BUFFER[2][2].batch_size = BATCH_SIZE

In [33]:
config = get_config(ENVS_REPLAY_BUFFER[2][0], COUNT_EPISODES, SEED, GAMMA, TAU, LEARNING_RATE, STEPS, HIDDEN_SIZE, ISDISCRETE)

In [34]:
env = train_DQN_FQI(config, ENVS_REPLAY_BUFFER[2][2])

[TRAIN 0] Mean Q Loss: 0.4492492973804474 
[TRAIN 1000] Mean Q Loss: 0.5127480052987303 
[TRAIN 2000] Mean Q Loss: 0.12332556456956081 
[TRAIN 3000] Mean Q Loss: 0.09930417562194634 
[TRAIN 4000] Mean Q Loss: 0.0873259709997219 
[TRAIN 5000] Mean Q Loss: 0.07213708964307443 
[TRAIN 6000] Mean Q Loss: 0.023183599393611077 
[TRAIN 7000] Mean Q Loss: 0.003996557657053927 
[TRAIN 8000] Mean Q Loss: 0.0034055667518114205 
[TRAIN 9000] Mean Q Loss: 0.003291155934566632 
[TRAIN 10000] Mean Q Loss: 0.0035289487834670582 
[TRAIN 11000] Mean Q Loss: 0.003461881785420701 
[TRAIN 12000] Mean Q Loss: 0.0034603459461941383 
[TRAIN 13000] Mean Q Loss: 0.0035855981232307387 
[TRAIN 14000] Mean Q Loss: 0.003544725531770382 
[TRAIN 15000] Mean Q Loss: 0.003975364172860282 
[TRAIN 16000] Mean Q Loss: 0.0032514701151958435 
[TRAIN 17000] Mean Q Loss: 0.0035457936262828296 
[TRAIN 18000] Mean Q Loss: 0.0037107690325792644 
[TRAIN 19000] Mean Q Loss: 0.0035439449009427334 
[TRAIN 20000] Mean Q Loss: 0.00339

In [35]:
env = ENVS_REPLAY_BUFFER[2][1]
model = MLP(env.observation_space.n, env.action_space.n, HIDDEN_SIZE)
model.load_state_dict(torch.load(f"trained_models/{ENVS_REPLAY_BUFFER[2][0]}FQI-DQN.pth"))
model.eval()  # Definir para modo de avaliação

MLP(
  (layers): Sequential(
    (0): Linear(in_features=48, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=4, bias=True)
  )
)

In [36]:
evaluate(config, env, model)

[Episódio 0]
[Episódio 1]
[Episódio 2]
[Episódio 3]
[Episódio 4]
[Episódio 5]
[Episódio 6]
[Episódio 7]
[Episódio 8]
[Episódio 9]
[Episódio 10]
[Episódio 11]
[Episódio 12]
[Episódio 13]
[Episódio 14]
[Episódio 15]
[Episódio 16]
[Episódio 17]
[Episódio 18]
[Episódio 19]
[Episódio 20]
[Episódio 21]
[Episódio 22]
[Episódio 23]
[Episódio 24]
[Episódio 25]
[Episódio 26]
[Episódio 27]
[Episódio 28]
[Episódio 29]
[Episódio 30]
[Episódio 31]
[Episódio 32]
[Episódio 33]
[Episódio 34]
[Episódio 35]
[Episódio 36]
[Episódio 37]
[Episódio 38]
[Episódio 39]
[Episódio 40]
[Episódio 41]
[Episódio 42]
[Episódio 43]
[Episódio 44]
[Episódio 45]
[Episódio 46]
[Episódio 47]
[Episódio 48]
[Episódio 49]
Média de recompensa após 50 episódios: -13.0


In [37]:
extract_and_display(model, ENVS_REPLAY_BUFFER[2][0])

## CartPole

In [38]:
COUNT_EPISODES = 50
SEED = 777
BATCH_SIZE = 256
STEPS = 50_000
GAMMA = 0.99  # Fator de desconto
TAU = 5e-3      # Taxa de atualização da target_network
LEARNING_RATE = 1e-4  # Taxa de aprendizado para o otimizador
HIDDEN_SIZE = [256, 128]
ISDISCRETE = False
# Extra:

In [39]:
ENVS_REPLAY_BUFFER[3][2].batch_size = BATCH_SIZE

In [40]:
config = get_config(ENVS_REPLAY_BUFFER[3][0], COUNT_EPISODES, SEED, GAMMA, TAU, LEARNING_RATE, STEPS, HIDDEN_SIZE, ISDISCRETE)

In [41]:
ENVS_REPLAY_BUFFER[3][2].memory

deque([Experience(state=[0.0444495715200901, -0.027678364887833595, 0.00578987505286932, 0.042084816843271255], action=np.int64(0), reward=1.0, done=False, next_state=[0.04389600455760956, -0.22288286685943604, 0.0066315713338553905, 0.33658885955810547]),
       Experience(state=[0.04389600455760956, -0.22288286685943604, 0.0066315713338553905, 0.33658885955810547], action=np.int64(0), reward=1.0, done=False, next_state=[0.03943834826350212, -0.4180985689163208, 0.013363348320126534, 0.6313556432723999]),
       Experience(state=[0.03943834826350212, -0.4180985689163208, 0.013363348320126534, 0.6313556432723999], action=np.int64(0), reward=1.0, done=False, next_state=[0.031076375395059586, -0.613404393196106, 0.02599046193063259, 0.9282169342041016]),
       Experience(state=[0.031076375395059586, -0.613404393196106, 0.02599046193063259, 0.9282169342041016], action=np.int64(1), reward=1.0, done=False, next_state=[0.018808288499712944, -0.4186427891254425, 0.04455479979515076, 0.643813

In [42]:
env = train_DQN_FQI(config, ENVS_REPLAY_BUFFER[3][2])

[TRAIN 0] Mean Q Loss: 0.6152109503746033 
[TRAIN 1000] Mean Q Loss: 0.0430274114847416 
[TRAIN 2000] Mean Q Loss: 0.09426255612657405 
[TRAIN 3000] Mean Q Loss: 0.17910099845239894 
[TRAIN 4000] Mean Q Loss: 0.260669478028547 
[TRAIN 5000] Mean Q Loss: 0.33377058083284644 
[TRAIN 6000] Mean Q Loss: 0.4119607457709499 
[TRAIN 7000] Mean Q Loss: 0.47639128504926337 
[TRAIN 8000] Mean Q Loss: 0.5694944225531071 
[TRAIN 9000] Mean Q Loss: 0.612375249738805 
[TRAIN 10000] Mean Q Loss: 0.670697148272302 
[TRAIN 11000] Mean Q Loss: 0.7457498992090114 
[TRAIN 12000] Mean Q Loss: 0.7972830304135569 
[TRAIN 13000] Mean Q Loss: 0.8430021826252341 
[TRAIN 14000] Mean Q Loss: 0.9005013006068766 
[TRAIN 15000] Mean Q Loss: 0.9385635730419308 
[TRAIN 16000] Mean Q Loss: 0.9613184188520536 
[TRAIN 17000] Mean Q Loss: 1.0180585996564477 
[TRAIN 18000] Mean Q Loss: 1.0349295388162136 
[TRAIN 19000] Mean Q Loss: 1.0788351855007932 
[TRAIN 20000] Mean Q Loss: 1.0819442419391125 
[TRAIN 21000] Mean Q Loss

In [43]:
env = ENVS_REPLAY_BUFFER[3][1]
model = MLP(env.observation_space.shape[0], env.action_space.n, HIDDEN_SIZE)
model.load_state_dict(torch.load(f"trained_models/{ENVS_REPLAY_BUFFER[3][0]}FQI-DQN.pth"))
model.eval()  # Definir para modo de avaliação

MLP(
  (layers): Sequential(
    (0): Linear(in_features=4, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=2, bias=True)
  )
)

In [44]:
evaluate_Continuous(config, env, model)

[Episódio 0]
[Episódio 1]
[Episódio 2]
[Episódio 3]
[Episódio 4]
[Episódio 5]
[Episódio 6]
[Episódio 7]
[Episódio 8]
[Episódio 9]
[Episódio 10]
[Episódio 11]
[Episódio 12]
[Episódio 13]
[Episódio 14]
[Episódio 15]
[Episódio 16]
[Episódio 17]
[Episódio 18]
[Episódio 19]
[Episódio 20]
[Episódio 21]
[Episódio 22]
[Episódio 23]
[Episódio 24]
[Episódio 25]
[Episódio 26]
[Episódio 27]
[Episódio 28]
[Episódio 29]
[Episódio 30]
[Episódio 31]
[Episódio 32]
[Episódio 33]
[Episódio 34]
[Episódio 35]
[Episódio 36]
[Episódio 37]
[Episódio 38]
[Episódio 39]
[Episódio 40]
[Episódio 41]
[Episódio 42]
[Episódio 43]
[Episódio 44]
[Episódio 45]
[Episódio 46]
[Episódio 47]
[Episódio 48]
[Episódio 49]
Média de recompensa após 50 episódios: 382.16


In [45]:
#extract_and_display(model, ENVS_REPLAY_BUFFER[3][0])
render_environment(model, ENVS_REPLAY_BUFFER[3][0])

Episódio 1: Recompensa = 389.0, Passos = 389
Episódio 2: Recompensa = 369.0, Passos = 369
Episódio 3: Recompensa = 375.0, Passos = 375
Episódio 4: Recompensa = 386.0, Passos = 386
Episódio 5: Recompensa = 378.0, Passos = 378


## LunarLander

In [46]:
COUNT_EPISODES = 50
SEED = 777
BATCH_SIZE = 256
STEPS = 50_000
GAMMA = 0.99  # Fator de desconto
TAU = 5e-3      # Taxa de atualização da target_network
LEARNING_RATE = 1e-4  # Taxa de aprendizado para o otimizador
HIDDEN_SIZE = [256, 256]
ISDISCRETE = False
# Extra:

In [47]:
ENVS_REPLAY_BUFFER[4][2].batch_size = BATCH_SIZE

In [48]:
config = get_config(ENVS_REPLAY_BUFFER[4][0], COUNT_EPISODES, SEED, GAMMA, TAU, LEARNING_RATE, STEPS, HIDDEN_SIZE, ISDISCRETE)

In [49]:
env = train_DQN_FQI(config, ENVS_REPLAY_BUFFER[4][2])

[TRAIN 0] Mean Q Loss: 1.131614327430725 
[TRAIN 1000] Mean Q Loss: 1.402336962044239 
[TRAIN 2000] Mean Q Loss: 1.1919699833393096 
[TRAIN 3000] Mean Q Loss: 1.1373762857019902 
[TRAIN 4000] Mean Q Loss: 1.1805349013209343 
[TRAIN 5000] Mean Q Loss: 1.1849224461913108 
[TRAIN 6000] Mean Q Loss: 1.2283229251503944 
[TRAIN 7000] Mean Q Loss: 1.2074924201071262 
[TRAIN 8000] Mean Q Loss: 1.189679004639387 
[TRAIN 9000] Mean Q Loss: 1.1863314238488674 
[TRAIN 10000] Mean Q Loss: 1.1762567874789238 
[TRAIN 11000] Mean Q Loss: 1.160461193025112 
[TRAIN 12000] Mean Q Loss: 1.1626529063284397 
[TRAIN 13000] Mean Q Loss: 1.190773769557476 
[TRAIN 14000] Mean Q Loss: 1.217473430544138 
[TRAIN 15000] Mean Q Loss: 1.198855661958456 
[TRAIN 16000] Mean Q Loss: 1.1915265872776508 
[TRAIN 17000] Mean Q Loss: 1.2058250352442264 
[TRAIN 18000] Mean Q Loss: 1.205084912329912 
[TRAIN 19000] Mean Q Loss: 1.1786380915641785 
[TRAIN 20000] Mean Q Loss: 1.1509545995593071 
[TRAIN 21000] Mean Q Loss: 1.17750

In [50]:
env = ENVS_REPLAY_BUFFER[4][1]
model = MLP(env.observation_space.shape[0], env.action_space.n, HIDDEN_SIZE)
model.load_state_dict(torch.load(f"trained_models/{ENVS_REPLAY_BUFFER[4][0]}FQI-DQN.pth"))
model.eval()  # Definir para modo de avaliação

MLP(
  (layers): Sequential(
    (0): Linear(in_features=8, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=4, bias=True)
  )
)

In [51]:
evaluate_Continuous(config, env, model)

[Episódio 0]
[Episódio 1]
[Episódio 2]
[Episódio 3]
[Episódio 4]
[Episódio 5]
[Episódio 6]
[Episódio 7]
[Episódio 8]
[Episódio 9]
[Episódio 10]
[Episódio 11]
[Episódio 12]
[Episódio 13]
[Episódio 14]
[Episódio 15]
[Episódio 16]
[Episódio 17]
[Episódio 18]
[Episódio 19]
[Episódio 20]
[Episódio 21]
[Episódio 22]
[Episódio 23]
[Episódio 24]
[Episódio 25]
[Episódio 26]
[Episódio 27]
[Episódio 28]
[Episódio 29]
[Episódio 30]
[Episódio 31]
[Episódio 32]
[Episódio 33]
[Episódio 34]
[Episódio 35]
[Episódio 36]
[Episódio 37]
[Episódio 38]
[Episódio 39]
[Episódio 40]
[Episódio 41]
[Episódio 42]
[Episódio 43]
[Episódio 44]
[Episódio 45]
[Episódio 46]
[Episódio 47]
[Episódio 48]
[Episódio 49]
Média de recompensa após 50 episódios: -1648.8908120960625


In [53]:
#extract_and_display(model, ENVS_REPLAY_BUFFER[3][0])
render_environment(model, ENVS_REPLAY_BUFFER[4][0])

Episódio 1: Recompensa = -36.11940081283462, Passos = 1000
Episódio 2: Recompensa = -41.626123798740224, Passos = 1000
Episódio 3: Recompensa = -0.5204568793972196, Passos = 1000
Episódio 4: Recompensa = -41.19399656712757, Passos = 1000
Episódio 5: Recompensa = -44.25570785648283, Passos = 1000
